In [28]:
import numpy as np 
import pandas as pd 
import os
train = pd.read_csv(r"C:\Users\90607\OneDrive\桌面\train.csv")
test = pd.read_csv(r"C:\Users\90607\OneDrive\桌面\test.csv")

In [29]:
print(train.head())
print(test.head())

   class                                              tweet
0      1  [9-1-13] 2:50 pm "son of a bitch ate my mac n ...
1      1  RT @BryceSerna: Don't be a pussy grab the boot...
2      2  RT @ClicquotSuave: bunch of rappers boutta flo...
3      2  @michigannews13 wow. Thats great language comi...
4      1  and this is why I'm single, I don't fuck with ...
   id                                              tweet
0   0  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
1   1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
2   2  !!!!!!"@__BrighterDays: I can not just sit up ...
3   3  !!!!&#8220;@selfiequeenbri: cause I'm tired of...
4   4  " @rhythmixx_ :hobbies include: fighting Maria...


In [22]:
train['class'] = train['class'].astype('category')

In [23]:
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [24]:
train['tweet'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in train['tweet']]
test['tweet'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in test['tweet']]

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train['tweet'],train['class'])

In [8]:
vect = TfidfVectorizer(ngram_range = (1,4)).fit(X_train)
vect_transformed_X_train = vect.transform(X_train)
vect_transformed_X_test = vect.transform(X_test)

In [10]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

In [11]:
modelSVC = SVC(C=100).fit(vect_transformed_X_train,y_train)
predictionsSVC = modelSVC.predict(vect_transformed_X_test)
sum(predictionsSVC==1),len(y_test),f1_score(y_test,predictionsSVC,average='micro')

(3232, 3718, 0.8679397525551372)

In [18]:
modelLR = LogisticRegression(C=100).fit(vect_transformed_X_train,y_train)
predictionsLR = modelLR.predict(vect_transformed_X_test)
sum(predictionsLR==1),len(y_test),f1_score(y_test,predictionsLR,average='micro')

c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(3062, 3718, 0.891877353415815)

In [13]:
modelNB = MultinomialNB(alpha=1.7).fit(vect_transformed_X_train,y_train)
predictionsNB = modelNB.predict(vect_transformed_X_test)
sum(predictionsNB==1),len(y_test),f1_score(y_test,predictionsNB,average='micro')

(3715, 3718, 0.7711135018827325)

In [14]:
modelRF = RandomForestClassifier(n_estimators=20).fit(vect_transformed_X_train,y_train)
predictionsRF = modelRF.predict(vect_transformed_X_test)
sum(predictionsRF==1),len(y_test),f1_score(y_test,predictionsRF,average='micro')

(3453, 3718, 0.8224852071005917)

In [15]:
modelSGD = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3).fit(vect_transformed_X_train,y_train)
predictionsSGD = modelSGD.predict(vect_transformed_X_test)
sum(predictionsSGD==1),len(y_test),f1_score(y_test,predictionsSGD,average='micro')

(3716, 3718, 0.7708445400753093)

Based on all the above models trained we conclude that the logistic regression(C=100) is the better model amoung them, ergo, we use this model as our final model.

In [16]:
vect = TfidfVectorizer(ngram_range = (1,4)).fit(train['tweet'])
vect_transformed_train = vect.transform(train['tweet'])
vect_transformed_test = vect.transform(test['tweet'])
FinalModel = LogisticRegression(C=100).fit(vect_transformed_train,train['class'])

c:\Users\90607\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
predictions = FinalModel.predict(vect_transformed_test)
submission = pd.DataFrame({'id':test['id'],'label':predictions})
submission.to_csv(r"C:\Users\90607\OneDrive\桌面\sample_upload.csv",index=False)